In [1]:
import numpy as np
import pandas as pd
import re

# GET RAW DATA

In [2]:
raw_data = pd.read_csv('raw_data.csv')

raw_data

,main,n_df
0,CENTRO DE EST...,0
1,DIREC.DE SIST.y COMUNICACIONES ...,0
2,* UNIDAD P.A.D.* ...,0
3,RESULTADOS GENE...,0
4,=============================================...,0
...,...,...
135174,0050 075546 ULLOA FLORES MANUEL JHAN FRAN ...,205
135175,0051 024879 CONTRERAS GUTIERREZ BRAYAN MERC...,205
135176,0052 013988 GARCIA CORONEL YOHAN ESMITH ...,205
135177,********************************************...,205


## DELETE AND REPLACE WHITE SPACES

In [3]:
many_white_spaces = re.compile(r'\s+')
one_white_space = ' '

raw_data['main'] = (
    raw_data['main']
    .str.replace(
        many_white_spaces,
        one_white_space,
        regex=True
    )
    .str.strip()
)

raw_data

,main,n_df
0,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,0
1,DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATI...,0
2,* UNIDAD P.A.D.* << TRUJILLO >> 12/08/2018,0
3,RESULTADOS GENERALES - ORDEN DE MERITO POR ESC...,0
4,==============================================...,0
...,...,...
135174,0050 075546 ULLOA FLORES MANUEL JHAN FRAN 20 8...,205
135175,0051 024879 CONTRERAS GUTIERREZ BRAYAN MERCEDE...,205
135176,0052 013988 GARCIA CORONEL YOHAN ESMITH 11 51 ...,205
135177,**********************************************...,205


# ONLY RESULTS

In [4]:
fourt_digits = re.compile(r'^\d{4}')

filter_by_character = (
    raw_data['main']
    .str.contains(fourt_digits, regex=True)
)

raw_results = raw_data[filter_by_character]

raw_results

,main,n_df
8,0001 171558 RODRIGUEZ SANCHEZ ROSICELA ELIZABE...,0
9,0002 061758 LLANOS SOLIS KIMBERLYN YEI 37.504 ...,0
10,0003 178758 ALVA PEREZ TAMARA ANTONELLA 48.918...,0
11,0004 028758 URIOL FLORES LIZ ROCIO 39.342 57.7...,0
12,0005 170858 VASQUEZ GUERRA CESAR IVAN 35.462 5...,0
...,...,...
135172,0048 033622 CASTILLO ROJAS HUGO GABRIEL 8 28 0...,205
135173,0049 013147 RUIZ JACOBO MERLY MILAGROS 7 27 0 ...,205
135174,0050 075546 ULLOA FLORES MANUEL JHAN FRAN 20 8...,205
135175,0051 024879 CONTRERAS GUTIERREZ BRAYAN MERCEDE...,205


# ONLY TITLES

In [5]:
raw_titles = raw_data[~filter_by_character]

raw_titles

,main,n_df
0,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,0
1,DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATI...,0
2,* UNIDAD P.A.D.* << TRUJILLO >> 12/08/2018,0
3,RESULTADOS GENERALES - ORDEN DE MERITO POR ESC...,0
4,==============================================...,0
...,...,...
135162,OR- RESPUESTAS BL PUNTAJE OBSERVA,205
135163,DEN CARNET APELLIDOS y NOMBRES BU MA DB AN TOT...,205
135164,==============================================...,205
135177,**********************************************...,205


# TRANSFORM TITLES

In [6]:
pag_number = re.compile(r'Pag\.\s\d+')
empty_string = ''

raw_titles.loc[:, 'main'] = (
    raw_titles['main']
    .str.replace(pag_number, empty_string, regex=True)
    .str.rstrip()
    )

raw_titles

,main,n_df
0,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,0
1,DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATI...,0
2,* UNIDAD P.A.D.* << TRUJILLO >> 12/08/2018,0
3,RESULTADOS GENERALES - ORDEN DE MERITO POR ESC...,0
4,==============================================...,0
...,...,...
135162,OR- RESPUESTAS BL PUNTAJE OBSERVA,205
135163,DEN CARNET APELLIDOS y NOMBRES BU MA DB AN TOT...,205
135164,==============================================...,205
135177,**********************************************...,205


In [7]:
titles_sep_sign = "="

def filter_group(dataframe_group):

    filter_title = (
        dataframe_group
        ['main']
        .str.startswith(titles_sep_sign)
    )

    index_value = (
        dataframe_group
        [filter_title]
        .index
        [0]
    )

    return dataframe_group.loc[:index_value - 1] # le puedes quitar el -1
    # para ver el = en el resultado

titles_group_by_n_df = (
    raw_titles
    .drop_duplicates(['main', 'n_df'])
    .groupby('n_df')
)

In [8]:
filtered_titles = map(lambda values: filter_group(values[1]), titles_group_by_n_df)
titles_before_equal_sign = pd.concat(filtered_titles)

titles_before_equal_sign.head(20)

,main,n_df
0,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,0
1,DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATI...,0
2,* UNIDAD P.A.D.* << TRUJILLO >> 12/08/2018,0
3,RESULTADOS GENERALES - ORDEN DE MERITO POR ESC...,0
3370,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,1
3371,DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATI...,1
3372,* UNIDAD P.A.D.* << JEQUETEPEQUE >> 12/08/2018,1
3373,RESULTADOS GENERALES - ORDEN DE MERITO POR ESC...,1
3678,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,2
3679,DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATI...,2


In [9]:
n_titles_per_df = titles_before_equal_sign.groupby("n_df").count()

n_titles_per_df

,main
n_df,
0,4
1,4
2,4
3,4
4,4
...,...
201,3
202,3
203,3


In [10]:
titles_before_equal_sign = pd.merge(titles_before_equal_sign, n_titles_per_df, left_on='n_df', right_on='n_df')

titles_before_equal_sign

,main_x,n_df,main_y
0,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,0,4
1,DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATI...,0,4
2,* UNIDAD P.A.D.* << TRUJILLO >> 12/08/2018,0,4
3,RESULTADOS GENERALES - ORDEN DE MERITO POR ESC...,0,4
4,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,1,4
...,...,...,...
716,17/03/2024 EXAMEN DE ADMISION ORDINARIO 2024-II,204,3
717,RESULTADOS GENERALES POR ESCUELA PROFESIONAL *...,204,3
718,OFICINA DE TECNOLOGIAS DE LA INFORMACION DE LA...,205,3
719,17/03/2024 EXAMEN DE ADMISION ORDINARIO 2024-II,205,3


In [11]:
def add_serie(df):
    max_value = df['main_y'].max()
    df['aux'] = range(max_value)
    return df

In [12]:
titles_with_sequence = map(lambda values: add_serie(values[1]), titles_before_equal_sign.groupby("n_df"))
titles_before_equal_sign = pd.concat(titles_with_sequence)

titles_before_equal_sign

,main_x,n_df,main_y,aux
0,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,0,4,0
1,DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATI...,0,4,1
2,* UNIDAD P.A.D.* << TRUJILLO >> 12/08/2018,0,4,2
3,RESULTADOS GENERALES - ORDEN DE MERITO POR ESC...,0,4,3
4,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,1,4,0
...,...,...,...,...
716,17/03/2024 EXAMEN DE ADMISION ORDINARIO 2024-II,204,3,1
717,RESULTADOS GENERALES POR ESCUELA PROFESIONAL *...,204,3,2
718,OFICINA DE TECNOLOGIAS DE LA INFORMACION DE LA...,205,3,0
719,17/03/2024 EXAMEN DE ADMISION ORDINARIO 2024-II,205,3,1


In [13]:
unstacked_titles = (
    titles_before_equal_sign
    .loc[:, ['n_df', 'aux', 'main_x']]
    .set_index(['n_df', 'aux'])
    .unstack()
    .droplevel(['aux'], axis=1)
)

unstacked_titles.columns = ['first', 'second', 'third', 'fourth']

unstacked_titles

,first,second,third,fourth
n_df,,,,
0,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATI...,* UNIDAD P.A.D.* << TRUJILLO >> 12/08/2018,RESULTADOS GENERALES - ORDEN DE MERITO POR ESC...
1,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATI...,* UNIDAD P.A.D.* << JEQUETEPEQUE >> 12/08/2018,RESULTADOS GENERALES - ORDEN DE MERITO POR ESC...
2,CENTRO DE ESTUDIOS PREUNIVERSITARIOS DE LA UNT...,DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATI...,* UNIDAD P.A.D.* << HUAMACHUCO >> 12/08/2018,RESULTADOS GENERALES - ORDEN DE MERITO POR ESC...
3,UNIVERSIDAD NACIONAL DE TRUJILLO - UNT,DIREC.DE SIST.y COMUNIC. EXAMEN DE ADMISION OR...,* UNIDAD P.A.D.* GRUPO : A 22/09/2018,RESULTADOS GENERALES POR ESCUELA PROFESIONAL
4,UNIVERSIDAD NACIONAL DE TRUJILLO - UNT,DIREC.DE SIST.y COMUNIC. EXAMEN DE ADMISION OR...,* UNIDAD P.A.D.* GRUPO : B 23/09/2018,RESULTADOS GENERALES POR ESCUELA PROFESIONAL
...,...,...,...,...
201,OFICINA DE TECNOLOGIAS DE LA INFORMACION DE LA...,16/03/2024 EXAMEN DE ADMISION ORDINARIO 2024-II,RESULTADOS GENERALES POR ESCUELA PROFESIONAL *...,NaN
202,OFICINA DE TECNOLOGIAS DE LA INFORMACION DE LA...,17/03/2024 EXAMEN DE ADMISION ORDINARIO 2024-II,RESULTADOS GENERALES POR ESCUELA PROFESIONAL *...,NaN
203,OFICINA DE TECNOLOGIAS DE LA INFORMACION DE LA...,16/03/2024 EXAMEN DE ADMISION ORDINARIO 2024-II,RESULTADOS GENERALES POR ESCUELA PROFESIONAL *...,NaN


## TRANSFORM SECOND COLUMN

In [14]:
unstacked_titles['second'].value_counts().sort_index(ascending=False).head(49)

second
OFI.DE TECNOLOG.DE LA INFORMAC. EXAMENES SUMATIVOS 2024- I - CEPUNT I                 3
OF.DE TECNOLOG.DE LA INFOR. EXAMEN DE ADMISION ORDINARIO 2024-I                       3
OF.DE TECNOLOG.DE LA INFOR. EXAMEN DE ADMISION ORDINARIO 2024- I - TRUJILLO           3
OF.DE TECNOLOG.DE LA INFOR. EXAMEN DE ADMISION ORDINARIO 2023-II - V A L L E          3
OF.DE TECNOLOG.DE LA INFOR. EXAMEN DE ADMISION ORDINARIO 2023-II - TRUJILLO           3
OF.DE TECNOLOG.DE LA INFOR. EXAMEN DE ADMISION ORDINARIO 2023-II - STGO.DE CHUCO      1
OF.DE TECNOLOG.DE LA INFOR. EXAMEN DE ADMISION ORDINARIO 2023-II - HUAMACHUCO         3
OF.DE TECNOLOG.DE LA INFOR. EXAMEN DE ADMISION ORDINARIO 2023-I                       3
OF.DE TECNOLOG.DE LA INFOR. EXAMEN DE ADMISION ORDINARIO 2023- I - TRUJILLO           3
OF.DE TECNOLOG.DE LA INFOR. EXAMEN DE ADMISION ORDINARIO 2022-II - V A L L E          3
OF.DE TECNOLOG.DE LA INFOR. EXAMEN DE ADMISION ORDINARIO 2022-II - TRUJILLO           3
OF.DE TECNOLOG.DE LA INFO

In [15]:
unstacked_titles['second'].value_counts().sort_index(ascending=False).tail(50)

second
DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATIVOS 2019-II - CEPUNT II                 3
DIREC.DE SIST.y COMUNICACIONES EXAMENES SUMATIVOS 2019- I - CEPUNT I                  3
DIREC.DE SIST.y COMUNICAC. EXAMEN DE ADMISION ORDINARIO 2021-II * STGO.DE CHUCO       1
DIREC.DE SIST.y COMUNICAC. EXAMEN DE ADMISION ORDINARIO 2020-II * STGO.DE CHUCO       1
DIREC.DE SIST.y COMUNICAC. EXAMEN DE ADMISION ORDINARIO 2019-II * STGO.DE CHUCO       1
DIREC.DE SIST.y COMUNIC. EXAMEN DE ADMISION ORDINARIO 2021-II - TRUJILLO              3
DIREC.DE SIST.y COMUNIC. EXAMEN DE ADMISION ORDINARIO 2021-II * JEQUETEPEQUE          3
DIREC.DE SIST.y COMUNIC. EXAMEN DE ADMISION ORDINARIO 2021- I - TRUJILLO              4
DIREC.DE SIST.y COMUNIC. EXAMEN DE ADMISION ORDINARIO 2021- I * TRUJILLO              4
DIREC.DE SIST.y COMUNIC. EXAMEN DE ADMISION ORDINARIO 2020-II - TRUJILLO              2
DIREC.DE SIST.y COMUNIC. EXAMEN DE ADMISION ORDINARIO 2020-II * JEQUETEPEQUE          2
DIREC.DE SIST.y COMUNIC. 

### SECOND COLUMN DATES

In [16]:
valid_date = re.compile(r'(\d\d/\d\d/\d+)')

second_date = (
    unstacked_titles
    ['second']
    .str.extract(valid_date)
    [0]
    # .value_counts()
    # .sum()
)

second_date.value_counts()

0
03/03/2024    7
17/03/2024    4
18/02/2024    3
16/03/2024    3
Name: count, dtype: int64

## TRANSFORM THIRD COLUMN

In [17]:
unstacked_titles['third'].value_counts().sort_index().head(60)

third
* AREA P.A.D.* << HUAMACHUCO >> 20/08/2023                                                           1
* AREA P.A.D.* << TRUJILLO >> 20/08/2023                                                             1
* AREA P.A.D.* << V A L L E >> 20/08/2023                                                            1
* AREA P.A.D.* AREA * A 30/09/2023                                                                   1
* AREA P.A.D.* AREA - D 23/09/2023                                                                   1
* AREA P.A.D.* AREA : A 07/04/2022                                                                   3
* AREA P.A.D.* AREA : A 21/10/2022                                                                   1
* AREA P.A.D.* AREA : A 24/03/2023                                                                   3
* AREA P.A.D.* AREA : A 28/10/2021                                                                   1
* AREA P.A.D.* AREA : B 08/04/2022                                 

In [18]:
unstacked_titles['third'].value_counts().sort_index().tail(60)

third
* UNIDAD P.A.D.* << JEQUETEPEQUE >> 12/08/2018                                             1
* UNIDAD P.A.D.* << JEQUETEPEQUE >> 18/08/2019                                             1
* UNIDAD P.A.D.* << TRUJILLO >> 09/02/2020                                                 1
* UNIDAD P.A.D.* << TRUJILLO >> 10/02/2019                                                 1
* UNIDAD P.A.D.* << TRUJILLO >> 12/08/2018                                                 1
* UNIDAD P.A.D.* << TRUJILLO >> 18/08/2019                                                 1
* UNIDAD P.A.D.* AREA : A 02/06/2021                                                       3
* UNIDAD P.A.D.* AREA : A 02/12/2020                                                       1
* UNIDAD P.A.D.* AREA : B 03/12/2020                                                       1
* UNIDAD P.A.D.* AREA : B 28/05/2021                                                       4
* UNIDAD P.A.D.* AREA : C 04/12/2020                            

### THIRD COLUMN DATES

In [19]:
nombre = 'third'

third_date = (
    unstacked_titles
    [nombre]
    .str.extract(valid_date)
    [0]
    # .value_counts()
    # .sum()
)

third_date.value_counts()

0
23/02/2020    7
24/02/2019    7
08/10/2021    6
11/02/2023    6
26/02/2022    6
             ..
21/09/2021    2
02/12/2020    2
03/12/2020    2
23/09/2023    2
21/11/2020    1
Name: count, Length: 63, dtype: int64

## JOIN COLUMN TITLES

In [20]:
# second_date
# second_users doesn't have users

# third_date

# third_tests doesn't have tests

# fourth_areas

In [21]:
date_title = (
    second_date
    .str.cat(third_date, na_rep=empty_string)
    .replace(empty_string, np.nan, regex=False)
 )

date_title.name = "fecha"
date_title

n_df
0      12/08/2018
1      12/08/2018
2      12/08/2018
3      22/09/2018
4      23/09/2018
          ...    
201    16/03/2024
202    17/03/2024
203    16/03/2024
204    17/03/2024
205    17/03/2024
Name: fecha, Length: 206, dtype: object

# TRANSFORM RESULTS

## NAMES

In [22]:
valid_name = re.compile(r'\s(\D+[0]?\D+[0]?)\s-?\d')

names = (
    raw_results
    ['main']
    .str.extract(valid_name)
    [0]
    .str.strip()
    .str.replace('0 0', '', regex=False)
    .str.rstrip()
    .str.replace('0', 'O', regex=False)
    .str.lower()
)

names

8         rodriguez sanchez rosicela elizabeth
9                   llanos solis kimberlyn yei
10                 alva perez tamara antonella
11                      uriol flores liz rocio
12                   vasquez guerra cesar ivan
                          ...                 
135172             castillo rojas hugo gabriel
135173              ruiz jacobo merly milagros
135174           ulloa flores manuel jhan fran
135175     contreras gutierrez brayan mercedes
135176             garcia coronel yohan esmith
Name: 0, Length: 115385, dtype: object

## GRADES

In [23]:
valid_grades = re.compile(r'(-?\d+\.\d+)')

grades = (
    raw_results
    ['main']
    .str.extractall(valid_grades)
    .unstack()
    .droplevel(level=0, axis=1)
)

grades

match,0,1,2,3,4
8,41.577,78.261,126.368,246.206,94.842
9,37.504,67.242,109.044,213.790,94.842
10,48.918,53.192,109.036,211.146,94.842
11,39.342,57.783,102.429,199.554,94.842
12,35.462,50.730,109.548,195.740,94.842
...,...,...,...,...,...
135172,4.044,NaN,NaN,NaN,NaN
135173,0.986,NaN,NaN,NaN,NaN
135174,-0.100,NaN,NaN,NaN,NaN
135175,-4.159,NaN,NaN,NaN,NaN


## SCHOOL AND DETAILS

In [24]:
after_grades = re.compile(r'(\.\d+\s[A-Z].*)')
digits = re.compile(r'\.\d+\s')
empty_string = ''

school_and_details = (
        raw_results
        ['main']
        .str.extract(after_grades)
        [0]
        .str.replace(digits, empty_string, regex=True)
)

school_and_details

8                SI ADMINISTRACION
9                SI ADMINISTRACION
10               SI ADMINISTRACION
11               SI ADMINISTRACION
12               SI ADMINISTRACION
                    ...           
135172    ING.AMBIENTAL NO INGRESA
135173    ING.AMBIENTAL NO INGRESA
135174    ING.AMBIENTAL NO INGRESA
135175    ING.AMBIENTAL NO INGRESA
135176    ING.AMBIENTAL NO INGRESA
Name: 0, Length: 115385, dtype: object

### DETAILS

In [25]:
valid_result = re.compile(r'(INGRESA.*|NO\sINGRESA.*|ING\.\s?2.*|AUSENTE.*|ANULADO.*|^SI\s|^NO\s|\sSI$|\sNO$)')
possitive_pattern = re.compile(r'INGRESA([\w\s-]+)?')
negative_pattern = re.compile(r'NO\D+')
positive_result = 'SI'
negative_result = 'NO'
ingresa = 'INGRESA'
no_ingresa = 'NO INGRESA'

details = (
        school_and_details
        .str.extract(valid_result)
        [0]
        .str.strip()
        .str.replace(negative_pattern, negative_result, regex=True)
        .str.replace(possitive_pattern, positive_result, regex=True)
        .str.replace(positive_result, ingresa, regex=False)
        .str.replace(negative_result, no_ingresa, regex=False)
        .str.replace('.', '', regex=False)
        .str.replace('ING 2-OPC', 'SEGUNDA OPC', regex=False)
        .str.lower()
        .str.strip()
)

details

8            ingresa
9            ingresa
10           ingresa
11           ingresa
12           ingresa
             ...    
135172    no ingresa
135173    no ingresa
135174    no ingresa
135175    no ingresa
135176    no ingresa
Name: 0, Length: 115385, dtype: object

In [26]:
details.value_counts()

0
no ingresa     95052
ingresa        15745
ausente         3003
segunda opc     1564
anulado           20
Name: count, dtype: int64

In [27]:
details[details.isnull()]

88692    NaN
Name: 0, dtype: object

### SCHOOL

In [28]:
valid_result2 = re.compile(r'INGRESA.*|NO\sINGRESA.*|ING\.\s?2.*|AUSENTE.*|ANULADO.*|^SI\s|^NO\s|\sSI$|\sNO$')

school = (
    school_and_details
    .str.replace(valid_result2, empty_string, regex=True)
    .str.upper()
    .str.strip()
    .str.replace(".", " ", regex=False)    
    .str.replace(":", " ", regex=False)   
    .str.strip()    
    .str.replace("\s+", " ", regex=True)   
    .str.lower()   
)

school

8         administracion
9         administracion
10        administracion
11        administracion
12        administracion
               ...      
135172     ing ambiental
135173     ing ambiental
135174     ing ambiental
135175     ing ambiental
135176     ing ambiental
Name: 0, Length: 115385, dtype: object

In [29]:
school.value_counts()

0
medicina              20453
derecho y cc pp        8464
ing industrial         7867
administracion         7553
enfermeria             6206
ing civil              5747
contabil y finan       5115
ing de sistemas        5084
arquitectura y urb     4540
trabajo social         3260
ing ambiental          2978
ing de minas           2859
farmacia y bbqq        2646
ing mecatronica        2462
economia               2401
ing agroindustrial     2117
informatica            1977
ing mecanica           1905
estomatologia          1892
educacion inicial      1884
ing quimica            1699
ccas comunicacion      1691
ed sec idiomas         1381
ccas biologicas        1265
microbiol y paras      1099
cc politica y gob       925
agronomia               925
ing metalurgica         859
educacion primaria      775
turismo                 743
ed sec lengua           697
ing agricola            632
ing agronomica          587
ing de materiales       573
derecho                 527
ed sec filosofia  

## JOIN RESULTS

In [30]:
column_results = ['nombres', 'r1', 'r2', 'r3', 'r4', 'r5', 'escuela', 'resultado', 'n_df']

results = pd.concat(
    [
        names,
        grades,
        school,
        details,
        raw_results['n_df']
    ],
    axis=1    
)

results.columns = column_results

results

,nombres,r1,r2,r3,r4,r5,escuela,resultado,n_df
8,rodriguez sanchez rosicela elizabeth,41.577,78.261,126.368,246.206,94.842,administracion,ingresa,0
9,llanos solis kimberlyn yei,37.504,67.242,109.044,213.790,94.842,administracion,ingresa,0
10,alva perez tamara antonella,48.918,53.192,109.036,211.146,94.842,administracion,ingresa,0
11,uriol flores liz rocio,39.342,57.783,102.429,199.554,94.842,administracion,ingresa,0
12,vasquez guerra cesar ivan,35.462,50.730,109.548,195.740,94.842,administracion,ingresa,0
...,...,...,...,...,...,...,...,...,...
135172,castillo rojas hugo gabriel,4.044,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205
135173,ruiz jacobo merly milagros,0.986,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205
135174,ulloa flores manuel jhan fran,-0.100,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205
135175,contreras gutierrez brayan mercedes,-4.159,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205


# JOIN TITLES WITH RESULTS

In [31]:
results_with_dates = pd.merge(
    results,
    date_title,
    left_on='n_df',
    right_index=True
)

results_with_dates

,nombres,r1,r2,r3,r4,r5,escuela,resultado,n_df,fecha
8,rodriguez sanchez rosicela elizabeth,41.577,78.261,126.368,246.206,94.842,administracion,ingresa,0,12/08/2018
9,llanos solis kimberlyn yei,37.504,67.242,109.044,213.790,94.842,administracion,ingresa,0,12/08/2018
10,alva perez tamara antonella,48.918,53.192,109.036,211.146,94.842,administracion,ingresa,0,12/08/2018
11,uriol flores liz rocio,39.342,57.783,102.429,199.554,94.842,administracion,ingresa,0,12/08/2018
12,vasquez guerra cesar ivan,35.462,50.730,109.548,195.740,94.842,administracion,ingresa,0,12/08/2018
...,...,...,...,...,...,...,...,...,...,...
135172,castillo rojas hugo gabriel,4.044,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024
135173,ruiz jacobo merly milagros,0.986,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024
135174,ulloa flores manuel jhan fran,-0.100,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024
135175,contreras gutierrez brayan mercedes,-4.159,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024


# READ DESCRIPTION DATA

In [32]:
description = pd.read_csv("description.csv")
description

,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
0,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,trujillo
1,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,valle jequetepeque
2,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,huamachuco
3,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ordinario,normal,trujillo
4,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ordinario,normal,trujillo
...,...,...,...,...,...,...
201,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,huamachuco
202,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,huamachuco
203,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,valle jequetepeque
204,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,valle jequetepeque


In [33]:
##############3
description['modalidad'].value_counts()

modalidad
normal                       140
excelencia                    27
5to secundaria                17
personas con discapacidad     11
victimas de la violencia       7
deportistas calificados        4
Name: count, dtype: int64

# JOIN RESULTS WITH DESCRIPTION

In [34]:
new_combined_data = (
    pd.merge(
        results_with_dates,
        description,
        left_on='n_df',
        right_index=True
        )
        )

new_combined_data

,nombres,r1,r2,r3,r4,r5,escuela,resultado,n_df,fecha,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
8,rodriguez sanchez rosicela elizabeth,41.577,78.261,126.368,246.206,94.842,administracion,ingresa,0,12/08/2018,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,trujillo
9,llanos solis kimberlyn yei,37.504,67.242,109.044,213.790,94.842,administracion,ingresa,0,12/08/2018,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,trujillo
10,alva perez tamara antonella,48.918,53.192,109.036,211.146,94.842,administracion,ingresa,0,12/08/2018,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,trujillo
11,uriol flores liz rocio,39.342,57.783,102.429,199.554,94.842,administracion,ingresa,0,12/08/2018,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,trujillo
12,vasquez guerra cesar ivan,35.462,50.730,109.548,195.740,94.842,administracion,ingresa,0,12/08/2018,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,trujillo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135172,castillo rojas hugo gabriel,4.044,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco
135173,ruiz jacobo merly milagros,0.986,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco
135174,ulloa flores manuel jhan fran,-0.100,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco
135175,contreras gutierrez brayan mercedes,-4.159,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco


# DELETE SOME GRADES

## MAIN GRADES ARE FOUND IN R4 COLUMN

In [35]:
filter_by_cepunt = new_combined_data['tipo_exam'] == 'CEPUNT'

In [36]:
filter_by_periodo = new_combined_data['fecha_periodo'].isin([2019, 2020])

In [37]:
main_grade4 = new_combined_data[(filter_by_cepunt) & (filter_by_periodo)]
main_grade4

,nombres,r1,r2,r3,r4,r5,escuela,resultado,n_df,fecha,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar


In [38]:
# execute only one time
main_grade4 = main_grade4.drop(['r1', 'r2', 'r3', 'r5'], axis=1)

main_grade4 = main_grade4.rename(columns={'r4': 'puntaje'})

main_grade4

,nombres,puntaje,escuela,resultado,n_df,fecha,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar


## MAIN GRADES ARE FOUND IN R1 COLUMN

In [39]:
filter_by_tipo = new_combined_data['tipo_exam'].isin(["extraordinario", "ordinario"])
filter_by_periodo2 = new_combined_data['fecha_periodo'] == 2024
filter_by_number2 = new_combined_data['numero_periodo'] == "II"


In [40]:
main_grade1 = new_combined_data[(filter_by_tipo) & (filter_by_periodo2) & (filter_by_number2)]

main_grade1

,nombres,r1,r2,r3,r4,r5,escuela,resultado,n_df,fecha,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
126006,castro garcia paulo gamaliel,147.835,NaN,NaN,NaN,NaN,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126007,ponce salvador naomi belen,126.434,NaN,NaN,NaN,NaN,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126008,pintado peña luciana nicolle,124.382,NaN,NaN,NaN,NaN,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126009,lezama diaz jennifer anghelina,124.377,NaN,NaN,NaN,NaN,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126010,abila villanueva adriana amparito,123.361,NaN,NaN,NaN,NaN,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135172,castillo rojas hugo gabriel,4.044,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco
135173,ruiz jacobo merly milagros,0.986,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco
135174,ulloa flores manuel jhan fran,-0.100,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco
135175,contreras gutierrez brayan mercedes,-4.159,NaN,NaN,NaN,NaN,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco


In [41]:
# execute only one time
main_grade1 = main_grade1.drop(['r2', 'r3', 'r4', 'r5'], axis=1)
main_grade1 = main_grade1.rename(columns={'r1': 'puntaje'})

main_grade1

,nombres,puntaje,escuela,resultado,n_df,fecha,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
126006,castro garcia paulo gamaliel,147.835,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126007,ponce salvador naomi belen,126.434,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126008,pintado peña luciana nicolle,124.382,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126009,lezama diaz jennifer anghelina,124.377,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126010,abila villanueva adriana amparito,123.361,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
...,...,...,...,...,...,...,...,...,...,...,...,...
135172,castillo rojas hugo gabriel,4.044,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco
135173,ruiz jacobo merly milagros,0.986,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco
135174,ulloa flores manuel jhan fran,-0.100,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco
135175,contreras gutierrez brayan mercedes,-4.159,ing ambiental,no ingresa,205,17/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ordinario,normal,santiago de chuco


## MAIN GRADES ARE FOUND IN R3 COLUMN

In [42]:
filter_general = (((filter_by_cepunt) & (filter_by_periodo)) | ((filter_by_tipo) & (filter_by_periodo2) & (filter_by_number2))) 

main_grade3 = new_combined_data[~filter_general]

In [43]:
# execute only one time
main_grade3 = main_grade3.drop(['r1', 'r2', 'r4', 'r5'], axis=1)
main_grade3 = main_grade3.rename(columns={'r3': 'puntaje'})
main_grade3

,nombres,puntaje,escuela,resultado,n_df,fecha,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
8,rodriguez sanchez rosicela elizabeth,126.368,administracion,ingresa,0,12/08/2018,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,trujillo
9,llanos solis kimberlyn yei,109.044,administracion,ingresa,0,12/08/2018,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,trujillo
10,alva perez tamara antonella,109.036,administracion,ingresa,0,12/08/2018,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,trujillo
11,uriol flores liz rocio,102.429,administracion,ingresa,0,12/08/2018,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,trujillo
12,vasquez guerra cesar ivan,109.548,administracion,ingresa,0,12/08/2018,http://admisionunt.info/docs/padrones/20191/20...,2019,I,cepunt,normal,trujillo
...,...,...,...,...,...,...,...,...,...,...,...,...
125992,lazaro llanos richard joel,73.996,ed sec historia,no ingresa,191,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo
125993,albujar viera luis alberto,56.455,ed sec historia,no ingresa,191,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo
125994,polo reyes julio martin,38.721,ed sec historia,no ingresa,191,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo
125995,alfaro medrano maria jose,37.298,ed sec historia,no ingresa,191,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo


## CONCAT GRADES

In [44]:
cleaned_results = pd.concat(
    [
        main_grade1,
        main_grade3,
        main_grade4
    ]
)
cleaned_results

,nombres,puntaje,escuela,resultado,n_df,fecha,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
126006,castro garcia paulo gamaliel,147.835,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126007,ponce salvador naomi belen,126.434,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126008,pintado peña luciana nicolle,124.382,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126009,lezama diaz jennifer anghelina,124.377,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
126010,abila villanueva adriana amparito,123.361,administracion,ingresa,192,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
...,...,...,...,...,...,...,...,...,...,...,...,...
125992,lazaro llanos richard joel,73.996,ed sec historia,no ingresa,191,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo
125993,albujar viera luis alberto,56.455,ed sec historia,no ingresa,191,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo
125994,polo reyes julio martin,38.721,ed sec historia,no ingresa,191,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo
125995,alfaro medrano maria jose,37.298,ed sec historia,no ingresa,191,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo


In [45]:
cleaned_results = cleaned_results.reset_index()
cleaned_results = cleaned_results.drop(['index', 'n_df'], axis=1)
cleaned_results

,nombres,puntaje,escuela,resultado,fecha,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
0,castro garcia paulo gamaliel,147.835,administracion,ingresa,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
1,ponce salvador naomi belen,126.434,administracion,ingresa,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
2,pintado peña luciana nicolle,124.382,administracion,ingresa,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
3,lezama diaz jennifer anghelina,124.377,administracion,ingresa,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
4,abila villanueva adriana amparito,123.361,administracion,ingresa,03/03/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,extraordinario,excelencia,trujillo
...,...,...,...,...,...,...,...,...,...,...,...
115380,lazaro llanos richard joel,73.996,ed sec historia,no ingresa,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo
115381,albujar viera luis alberto,56.455,ed sec historia,no ingresa,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo
115382,polo reyes julio martin,38.721,ed sec historia,no ingresa,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo
115383,alfaro medrano maria jose,37.298,ed sec historia,no ingresa,18/02/2024,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,cepunt,normal,trujillo


In [46]:
cleaned_results['modalidad'].value_counts()

modalidad
normal                       101105
5to secundaria                 8709
excelencia                     5438
personas con discapacidad       104
victimas de la violencia         24
deportistas calificados           5
Name: count, dtype: int64

In [47]:
cleaned_results['puntaje'].isna().sum()

0